In [2]:
import numpy as np
import random
import copy

# ===========================
# ==== Funções do ambiente ===
# ===========================

class Celula:
    def __init__(self):
        self.bloco = 0  # 1: tesouro, 2: monstro, 4: buraco, 8: fedor, 16: brisa, 32: inicio

def criacao(matriz, linhas, colunas):
    for i in range(linhas):
        linha = []
        for j in range(colunas):
            linha.append(Celula())
        matriz.append(linha)

    # Definir posição inicial
    matriz[linhas-1][0].bloco += 32

    # Inserir tesouro
    matriz[random.randint(0, linhas-2)][random.randint(1, colunas-1)].bloco += 1

    # Inserir monstro
    x, y = random.randint(0, linhas-1), random.randint(0, colunas-1)
    while (x == linhas-1 and y == 0) or (matriz[x][y].bloco & 1):
        x, y = random.randint(0, linhas-1), random.randint(0, colunas-1)
    matriz[x][y].bloco += 2

    # Adicionar fedor ao redor do monstro
    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
      nx, ny = x+dx, y+dy
      if 0 <= nx < linhas and 0 <= ny < colunas:
          matriz[nx][ny].bloco |= 8  # Ativa o bit de fedor
    # Inserir buracos
    for _ in range(3):
        x, y = random.randint(0, linhas-1), random.randint(0, colunas-1)
        while (x == linhas-1 and y == 0) or (matriz[x][y].bloco & 1) or (matriz[x][y].bloco & 2):
            x, y = random.randint(0, linhas-1), random.randint(0, colunas-1)
        matriz[x][y].bloco += 4

   # Adicionar brisa ao redor dos buracos
    for i in range(linhas):
        for j in range(colunas):
            if matriz[i][j].bloco & 4:
                for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                    nx, ny = i+dx, j+dy
                    if 0 <= nx < linhas and 0 <= ny < colunas:
                        matriz[nx][ny].bloco |= 16  # Ativa o bit de brisa

def mostrar_desenvolvedor(matriz):
    print('Mapa (modo desenvolvedor):')
    for linha in matriz:
        texto = ''
        for cel in linha:
            texto += f'{cel.bloco:02d} '
        print(texto)
    print()

def dfs_verifica_caminho(matriz, linhas, colunas, posicao, visitados=None):
    if visitados is None:
        visitados = set()
    x, y = posicao
    if not (0 <= x < linhas and 0 <= y < colunas):
        return False
    if (x, y) in visitados:
        return False
    if matriz[x][y].bloco & 4 or matriz[x][y].bloco & 2:
        return False
    if matriz[x][y].bloco & 1:
        return True

    visitados.add((x, y))
    for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
        if dfs_verifica_caminho(matriz, linhas, colunas, (x+dx, y+dy), visitados):
            return True
    return False


# ===========================
# ======= Q-Learning ========
# ===========================

# Configuração do mapa
mapa_x = 5
mapa_y = 5
mapa = []
criacao(mapa, mapa_x, mapa_y)

# Fixar um mapa viável
while not dfs_verifica_caminho(mapa, mapa_x, mapa_y, [mapa_x-1, 0]):
    mapa.clear()
    criacao(mapa, mapa_x, mapa_y)

# Mostrar o mapa fixo
print("Mapa fixo para o Q-Learning:")
mostrar_desenvolvedor(mapa)

# Definir ações (0: cima, 1: baixo, 2: esquerda, 3: direita)
acoes = [(-1,0), (1,0), (0,-1), (0,1)]
n_acoes = len(acoes)

# Inicializar Q-table
Q = np.zeros((mapa_x, mapa_y, n_acoes))

# Hiperparâmetros
alpha = 0.1
gamma = 0.99
epsilon = 0.1

# Função para escolher ação (ε-greedy)
def escolher_acao(estado):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, n_acoes - 1)  # Exploração
    else:
        return np.argmax(Q[estado[0], estado[1]])  # Exploração

# Função de recompensa
def recompensa(jogo, pos, status):
    x, y = pos
    bloco = jogo[x][y].bloco

    if bloco & 4 or bloco & 2:  # Buraco ou monstro
        return -1000, True  # Derrota

    if bloco & 1 and status[0] == 1:
        status[0] = 2  # Pegou tesouro
        jogo[x][y].bloco -= 1
        return 1000, False

    if (bloco & 32) and status[0] == 2:
        status[0] = 0  # Devolveu o tesouro
        return 2000, True  # Vitória

    return -1, False  # Custo de andar

# Função de movimento
def mover(estado, acao):
    dx, dy = acoes[acao]
    nx = estado[0] + dx
    ny = estado[1] + dy

    if 0 <= nx < mapa_x and 0 <= ny < mapa_y:
        return (nx, ny)
    else:
        return estado  # Bate na parede

# ===========================
# ===== Q-Learning Loop =====
# ===========================

vitorias = 0
episodio = 0

while vitorias < 2:
    episodio += 1
    # Clonar o mapa para o episódio
    jogo = copy.deepcopy(mapa)

    pos = (mapa_x-1, 0)  # Posição inicial
    status = [1, 1]  # status[0]=tesouro (1 buscar, 2 carregando), status[1]=monstro (não usado)

    total_recompensa = 0

    for passo in range(1000):  # Limite de passos
        acao = escolher_acao(pos)
        novo_pos = mover(pos, acao)

        r, done = recompensa(jogo, novo_pos, status)

        total_recompensa += r

        Q[pos[0], pos[1], acao] += alpha * (
            r + gamma * np.max(Q[novo_pos[0], novo_pos[1]]) - Q[pos[0], pos[1], acao]
        )

        pos = novo_pos

        if done:
            if r > 0:
                vitorias += 1
                print(f"✅ Vitória {vitorias} no episódio {episodio} com recompensa {total_recompensa}")
            else:
                print(f"❌ Derrota no episódio {episodio} com recompensa {total_recompensa}")
            break

print("\n🏆 Treinamento finalizado! O agente venceu duas vezes.\n")
print("Tabela Q final:")
print(Q)


Mapa fixo para o Q-Learning:
Mapa (modo desenvolvedor):
00 16 04 16 00 
16 00 17 00 00 
04 16 00 08 00 
16 16 08 02 08 
48 04 16 08 00 

❌ Derrota no episódio 1 com recompensa -1001
❌ Derrota no episódio 2 com recompensa -1002
❌ Derrota no episódio 3 com recompensa -1014
❌ Derrota no episódio 4 com recompensa -13
❌ Derrota no episódio 5 com recompensa -1015
❌ Derrota no episódio 6 com recompensa -1008
❌ Derrota no episódio 7 com recompensa -1002
❌ Derrota no episódio 8 com recompensa -1002
❌ Derrota no episódio 9 com recompensa -1009
❌ Derrota no episódio 10 com recompensa -1005
❌ Derrota no episódio 11 com recompensa -177
❌ Derrota no episódio 12 com recompensa -105
❌ Derrota no episódio 13 com recompensa -6
❌ Derrota no episódio 14 com recompensa -32
❌ Derrota no episódio 15 com recompensa -18
❌ Derrota no episódio 16 com recompensa -42
❌ Derrota no episódio 17 com recompensa -24
❌ Derrota no episódio 18 com recompensa -55
❌ Derrota no episódio 19 com recompensa -106
❌ Derrota no epi